## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,59.29,27,1,3.00,clear sky
1,1,Jamestown,US,42.0970,-79.2353,57.02,30,20,17.27,few clouds
2,2,Lebu,CL,-37.6167,-73.6500,57.90,70,69,7.14,broken clouds
3,3,Ukiah,US,39.1502,-123.2078,41.38,95,100,0.00,overcast clouds
4,4,Kapaa,US,22.0752,-159.3190,71.92,80,40,16.11,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Convert the input to a floating-point decimal number by
# wrapping the input() statement with the float() method, as float(input())
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapaa,US,22.0752,-159.3190,71.92,80,40,16.11,scattered clouds
7,7,Port Alfred,ZA,-33.5906,26.8910,64.18,80,100,4.00,overcast clouds
8,8,Basco,PH,20.4487,121.9702,76.86,78,66,24.76,broken clouds
10,10,Sibolga,ID,1.7427,98.7792,79.02,86,99,2.17,heavy intensity rain
13,13,Puerto Ayora,EC,-0.7393,-90.3518,78.76,81,48,10.47,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
681,681,Santa Maria,BR,-29.6842,-53.8069,64.11,56,0,5.30,clear sky
682,682,Naron,ES,43.5167,-8.1528,68.56,66,4,6.53,clear sky
684,684,Mogadishu,SO,2.0371,45.3438,83.30,74,97,14.52,overcast clouds
686,686,Margosatubig,PH,7.5777,123.1658,78.04,86,43,2.48,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new empty column "Hotel Name"
# fill the cells with NaN instead of "", because ""is not empty.
import numpy as np

hotel_df["Hotel Name"] = np.nan

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,71.92,scattered clouds,22.0752,-159.3190,NaN
7,Port Alfred,ZA,64.18,overcast clouds,-33.5906,26.8910,NaN
8,Basco,PH,76.86,broken clouds,20.4487,121.9702,NaN
10,Sibolga,ID,79.02,heavy intensity rain,1.7427,98.7792,NaN
13,Puerto Ayora,EC,78.76,scattered clouds,-0.7393,-90.3518,NaN
15,Cabo San Lucas,MX,72.52,overcast clouds,22.8909,-109.9124,NaN
16,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,NaN
19,Rikitea,PF,77.61,light rain,-23.1203,-134.9692,NaN
27,Parana,AR,69.57,clear sky,-31.7320,-60.5238,NaN
28,Salalah,OM,84.29,clear sky,17.0151,54.0924,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to "location" key for the params dictionary.
    # using the f-string format

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        # retrieve the "name" of the first hotel in the "results" dictionary
        # by using code:     hotels["results"][0]["name"]
        # then put in hotel_df["Hotel Name"] column for each city(row)
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(30)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,71.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Port Alfred,ZA,64.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel
8,Basco,PH,76.86,broken clouds,20.4487,121.9702,Nathaniel's Lodge
10,Sibolga,ID,79.02,heavy intensity rain,1.7427,98.7792,Hotel Syariah CN Darussalam
13,Puerto Ayora,EC,78.76,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Cabo San Lucas,MX,72.52,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
16,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Rikitea,PF,77.61,light rain,-23.1203,-134.9692,People ThankYou
27,Parana,AR,69.57,clear sky,-31.7320,-60.5238,Howard Johnson by Wyndham Plaza Hotel Mayorazgo
28,Salalah,OM,84.29,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [9]:
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             25
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,71.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Port Alfred,ZA,64.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel
8,Basco,PH,76.86,broken clouds,20.4487,121.9702,Nathaniel's Lodge
10,Sibolga,ID,79.02,heavy intensity rain,1.7427,98.7792,Hotel Syariah CN Darussalam
13,Puerto Ayora,EC,78.76,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Cabo San Lucas,MX,72.52,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
16,Mahebourg,MU,79.09,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Rikitea,PF,77.61,light rain,-23.1203,-134.9692,People ThankYou
27,Parana,AR,69.57,clear sky,-31.7320,-60.5238,Howard Johnson by Wyndham Plaza Hotel Mayorazgo
28,Salalah,OM,84.29,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [11]:
clean_hotel_df.count()

City                   359
Country                359
Max Temp               359
Current Description    359
Lat                    359
Lng                    359
Hotel Name             359
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file= "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt>
    <dd>{Hotel Name}</dd>
    <dt>City</dt>
    <dd>{City}</dd>
    <dt>Country</dt>
    <dd>{Country}</dd>
    <dt>Current Description</dt>
    <dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))